In [1]:
import boto3
from botocore.exceptions import ClientError
import time

class BedrockAgentService:
    """
    A service class to interact with AWS Bedrock Agents.
    
    This class provides functionality to invoke Bedrock agents and handle their responses,
    including parameter parsing and control flow management.
    """
    
    def __init__(self, agent_id: str, alias_id: str = "TSTALIASID", session_id = None, client = None) -> None:
        """
        Initialize the BedrockAgentService.
        
        Args:
            agent_id (str): The ID of the Bedrock agent to interact with
            alias_id (str, optional): The alias ID for the agent. Defaults to "TSTALIASID"
            client: The boto3 client for Bedrock runtime. If None, creates a new client
        """
        self.agents_runtime_client = (
            client if client else boto3.client("bedrock-agent-runtime")
        )
        self.agent_id = agent_id
        self.alias_id = alias_id
        self.session_id = session_id or str(int(time.time()))

    def set_session_id(self, session_id: str) -> None:
        """
        Set the session ID for the agent.

        Args:
            session_id (str): The session ID to be used for the agent interaction
        """
        self.session_id = session_id

    def parse_parameters(self, parameters: list) -> dict:
        """
        Parse parameters from the agent response into a dictionary format.
        
        Args:
            parameters (list): List of parameter dictionaries containing name and value pairs
            
        Returns:
            dict: Dictionary mapping parameter names to their values
        """
        return {param["name"]: param["value"] for param in parameters}

    def return_control(self, completion: dict) -> dict:
        """
        Process the completion response and extract control flow information.
        
        Args:
            completion (dict): The completion response from the agent containing control information
            
        Returns:
            dict: Processed control information including invocation details and parameters
        """
        rc = completion.get("returnControl", {})
        invocationId = rc.get("invocationId", "")
        invocationInputs = rc.get("invocationInputs",[])

        return_dict = { "invocationId": invocationId}
        
        if len(invocationInputs):
            functionInvocationInput = invocationInputs[0].get("functionInvocationInput", {})
            actionGroup = functionInvocationInput.get("actionGroup", {})
            agentId = functionInvocationInput.get("agentId", None)
            actionInvocationType = functionInvocationInput.get("actionInvocationType", "")
            functionName = functionInvocationInput.get("function", "")
            parameters = functionInvocationInput.get("parameters", {})
            parsed_parameters = self.parse_parameters(parameters)
            return_dict.update(
                {
                    "actionGroup": actionGroup,
                    "agentId": agentId,
                    "functionName": functionName,
                    "parameters": parsed_parameters,
                    "actionInvocationType": actionInvocationType,
                }
            )
        return return_dict

    def return_control_invocation_results(
        self, 
        invocation_id: str, 
        action_group: str, 
        function_name: str, 
        invocation_result: str,
        agent_id : str  = None,
        session_id: str = None, 
    ) -> str:
        """
        Send the results of a function invocation back to the agent.
        
        Args:
            session_id (str): The current session identifier
            invocation_id (str): The ID of the invocation being responded to
            action_group (str): The action group that was invoked
            function_name (str): The name of the function that was called
            invocation_result (str): The result of the function invocation
            
        Returns:
            str: The completion response from the agent after processing the results
        """
        try:
            response = self.agents_runtime_client.invoke_agent(
                agentId=self.agent_id,
                agentAliasId=self.alias_id,
                sessionId=session_id or self.session_id,
                sessionState={
                    "invocationId": invocation_id,
                    "returnControlInvocationResults": [
                        {
                            "functionResult": {
                                "actionGroup": action_group,
                                "agentId": agent_id or self.agent_id,
                                "function": function_name,
                                "responseBody": {"TEXT": {"body": invocation_result}},
                            }
                        }
                    ],
                },
            )

            completion = ""

            for event in response.get("completion"):
                if event.get("returnControl"):
                    return self.return_control(event)
                chunk = event["chunk"]
                completion = completion + chunk["bytes"].decode()

        except ClientError as e:
            print(f"Couldn't invoke agent. {e}")

        return completion

    # from https://docs.aws.amazon.com/code-library/latest/ug/python_3_bedrock-agent-runtime_code_examples.html
    def invoke_agent(self, prompt: str, session_id: str = None) -> dict:
        """
        Invoke the Bedrock agent with a prompt and handle the response.
        
        Args:
            session_id (str): The session identifier for the conversation
            prompt (str): The user's input prompt to send to the agent
            
        Returns:
            dict: The processed response from the agent including any control flow modifications
        """

        kwargs = dict(
            agentId=self.agent_id,
            agentAliasId=self.alias_id,
            sessionId=session_id or self.session_id,
            inputText=prompt
        )
        if prompt.startswith("/new "):            
            kwargs["inputText"] = prompt.replace("/new ", "")
            kwargs["endSession"] = True

        print ("AGENT INVOCATION KWARGS")
        print (kwargs)
        try:
            response = self.agents_runtime_client.invoke_agent(**kwargs, )

            completion = ""

            for event in response.get("completion"):
                if event.get("returnControl"):
                    return event, self.return_control(event)
                chunk = event["chunk"]
                completion = completion + chunk["bytes"].decode()

        except ClientError as e:
            print(f"Couldn't invoke agent. {e}")
            return None

        return completion


In [2]:
AGENT_ID = "ZY07IWCMGN"
session_id = "10003"
bedrock_agent = BedrockAgentService(agent_id=AGENT_ID, session_id=session_id)

In [3]:
bedrock_agent.invoke_agent("Hola tengo un problema con mi pedido")

AGENT INVOCATION KWARGS
{'agentId': 'ZY07IWCMGN', 'agentAliasId': 'TSTALIASID', 'sessionId': '10003', 'inputText': 'Hola tengo un problema con mi pedido'}


'Entiendo que tiene un problema con su pedido. ¿Podría proporcionarme el número de pedido y su número de documento de identidad para que pueda ayudarle a verificar el estado de su orden?'

In [4]:
bedrock_agent.invoke_agent("el pedido 10020030 no ha llegado, rut 12345678-9")

AGENT INVOCATION KWARGS
{'agentId': 'ZY07IWCMGN', 'agentAliasId': 'TSTALIASID', 'sessionId': '10003', 'inputText': 'el pedido 10020030 no ha llegado, rut 12345678-9'}


'Según nuestro sistema, su pedido número 10020030 actualmente está en estado "Despacho Programado". Está programado para ser entregado el 6 de agosto de 2024 en la dirección Calle las acacias 123, Providencia, Región Metropolitana. Si necesita más información o tiene alguna consulta adicional, por favor no dude en preguntar.'

In [5]:
bedrock_agent.invoke_agent("Pero es que ya pasó la fecha de envío, cuántos dias?")

AGENT INVOCATION KWARGS
{'agentId': 'ZY07IWCMGN', 'agentAliasId': 'TSTALIASID', 'sessionId': '10003', 'inputText': 'Pero es que ya pasó la fecha de envío, cuántos dias?'}


'Entiendo su preocupación. Según nuestros registros, su pedido está 220 días atrasado con respecto a la fecha de entrega programada originalmente (6 de agosto de 2024). Le recomiendo comunicarse con nuestro servicio al cliente para investigar el motivo del retraso y encontrar una solución. ¿Le gustaría que le proporcione más información o ayuda para resolver este problema?'

In [6]:
bedrock_agent.invoke_agent("que puedo hacer?")

AGENT INVOCATION KWARGS
{'agentId': 'ZY07IWCMGN', 'agentAliasId': 'TSTALIASID', 'sessionId': '10003', 'inputText': 'que puedo hacer?'}


'He creado un ticket de soporte con el número 202503142209 para investigar el problema con su pedido. Nuestro equipo de servicio al cliente se pondrá en contacto con usted pronto para:\n\n1. Investigar el motivo del retraso de su pedido\n2. Buscar una solución\n3. Ofrecerle una compensación o reembolso si es necesario\n\nLe recomiendo estar atento a cualquier comunicación de nuestra parte. ¿Hay algún método de contacto que prefiera (teléfono, correo electrónico)?'

In [7]:
bedrock_agent.invoke_agent("puedes darme el estado del ticket 202503142209? ")

AGENT INVOCATION KWARGS
{'agentId': 'ZY07IWCMGN', 'agentAliasId': 'TSTALIASID', 'sessionId': '10003', 'inputText': 'puedes darme el estado del ticket 202503142209? '}


'El estado actual del ticket 202503142209 es "open" (abierto). Esto significa que:\n- El ticket está activo en nuestro sistema\n- Nuestro equipo de servicio al cliente está trabajando en su caso\n- Aún no se ha resuelto completamente la situación con su pedido\n\nLe recomendamos estar atento a futuras comunicaciones relacionadas con este ticket.'

In [8]:
bedrock_agent.invoke_agent("Necesito escalarlo")

AGENT INVOCATION KWARGS
{'agentId': 'ZY07IWCMGN', 'agentAliasId': 'TSTALIASID', 'sessionId': '10003', 'inputText': 'Necesito escalarlo'}


({'returnControl': {'invocationId': '7b75a175-b01a-48f6-b3fa-5abbbd35149d',
   'invocationInputs': [{'functionInvocationInput': {'actionGroup': 'Escalation',
      'actionInvocationType': 'RESULT',
      'agentId': 'ORXFDFEY5N',
      'collaboratorName': 'escalation',
      'function': 'escalate',
      'parameters': [{'name': 'identity_document_number',
        'type': 'string',
        'value': '12345678-9'},
       {'name': 'order_number', 'type': 'string', 'value': '10020030'},
       {'name': 'description',
        'type': 'string',
        'value': 'Pedido con 220 días de retraso, cliente solicita escalamiento urgente para resolver problema de entrega'},
       {'name': 'ticket_number',
        'type': 'string',
        'value': '202503142209'}]}}]}},
 {'invocationId': '7b75a175-b01a-48f6-b3fa-5abbbd35149d',
  'actionGroup': 'Escalation',
  'agentId': 'ORXFDFEY5N',
  'functionName': 'escalate',
  'parameters': {'identity_document_number': '12345678-9',
   'order_number': '1002003

In [9]:
bedrock_agent.return_control_invocation_results(
    "7b75a175-b01a-48f6-b3fa-5abbbd35149d",
    "Escalation",
    "escalate",
    "escalation=OK",
    "ORXFDFEY5N",
)

'He escalado su caso a nuestro equipo especializado. El escalamiento ha sido exitoso y será revisado por un supervisor. Le recomendamos esperar una respuesta en las próximas 24-48 horas hábiles. Mantendremos su ticket 202503142209 con prioridad alta para resolver el problema con su pedido 10020030.'

In [10]:
bedrock_agent.invoke_agent("OK Gracias, por tu trabajo")

AGENT INVOCATION KWARGS
{'agentId': 'ZY07IWCMGN', 'agentAliasId': 'TSTALIASID', 'sessionId': '10003', 'inputText': 'OK Gracias, por tu trabajo'}


'De nada. Estoy aquí para ayudarle en lo que necesite. ¿En qué puedo servirle hoy?'